<a href="https://colab.research.google.com/github/hmikkos/Voxnet/blob/master/LoadModelTestModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import os
from google.colab import drive
drive.mount('/content/drive')
root_path = '/content/drive/My Drive/Projet MODAL'
import os
os.chdir(root_path)  #change dir
num_classes=10
import io
import numpy as np
import torch
import torch.utils.data
import random
from random import shuffle
def importer():
  FichierTrain = 'test2.txt'
  Fichier = open(FichierTrain,'r') 
  cuda=True
  test=[]
  string=Fichier.readline()
  labels=[]
  testlabel=[]
  while True:
      #new matrix
      if len(string)==3:
          label=int(string[1])
          if (label in labels)==False:
              print(label)
          labels.append(label)
      elif len(string)==4 : 
          label=int(string[1])*10+int(string[2])
          if (label in labels)==False:
              print(label)
          labels.append(label)
      temp=np.zeros((32,32,32))
      string=Fichier.readline()
    
      if label==num_classes:
          break
    
      while string[0]!='#':
          s=string
          string=Fichier.readline()
          if s[1]==',':
            x=int(s[0])
            s=s[1:]
            s=s[1:]
          else :
            x=int(s[0])*10+int(s[1])
            s=s[1:]
            s=s[1:]
            s=s[1:]
          if s[1]==',':
            y=int(s[0])
            s=s[1:]
            s=s[1:]
          else :
            y=int(s[0])*10+int(s[1])
            s=s[1:]
            s=s[1:]
            s=s[1:]
          if len(s)==2:
            z=int(s[0])
          else:
            z=int(s[0])*10+int(s[1])
          temp[x+1,y+1,z+1]=1  
      c=random.uniform(0, 1)
      if c<0.7:
        test.append(temp)
        testlabel.append(label)
  Fichier.close()
  test = np.asarray(test)
  testlabel=np.asarray(testlabel)
  test= torch.from_numpy(test)
  tensor_target = torch.from_numpy(testlabel)
  testset = torch.utils.data.TensorDataset(test, tensor_target)
  return(testset)
from collections import OrderedDict
class VoxNet(torch.nn.Module):

    def __init__(self, num_classes, input_shape=(32, 32, 32),
                 weights_path=None,
                 load_body_weights=False,
                 load_head_weights=False):
        super(VoxNet, self).__init__()
        dictionnaire=OrderedDict([
            ('conv1', torch.nn.Conv3d(in_channels=1,
                                      out_channels=32, kernel_size=5, stride=2)),
            ('lkrelu1', torch.nn.LeakyReLU()),
            ('drop1', torch.nn.Dropout(p=0.2)),
            ('conv2', torch.nn.Conv3d(in_channels=32, out_channels=32, kernel_size=3)),
            ('lkrelu2', torch.nn.LeakyReLU()),
            ('pool2', torch.nn.MaxPool3d(2)),
            ('drop2', torch.nn.Dropout(p=0.3))
        ])
        self.body = torch.nn.Sequential(dictionnaire)

        # Trick to accept different input shapes
        x = self.body(torch.autograd.Variable(
            torch.rand((1, 1) + input_shape)))
        first_fc_in_features = 1
        for n in x.size()[1:]:
            first_fc_in_features *= n

        self.head = torch.nn.Sequential(OrderedDict([
            ('fc1', torch.nn.Linear(first_fc_in_features, 128)),
            ('relu1', torch.nn.ReLU()),
            ('drop3', torch.nn.Dropout(p=0.4)),
            ('fc2', torch.nn.Linear(128, num_classes))
        ]))


    def forward(self, x):
        x=x.view(x.size()[0],1,32,32,32)
        x = self.body(x)
        x = x.view(x.size(0), -1)
        x = self.head(x)
        return x
cuda=True    
def accuracy(net, test_loader, cuda=cuda):
  net.eval()
  correct = 0
  total = 0
  loss = 0
  with torch.no_grad():
      for data in test_loader:
          images, labels = data
          if cuda:
            images = images.type(torch.cuda.FloatTensor)
            labels = labels.type(torch.cuda.LongTensor)
          outputs = net(images)
          # loss+= criterion(outputs, labels).item()
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()
          # if total > 100:
            # break
  net.train()
  print('Accuracy of the network on the loader : %d %%' % (
      100 * correct / total))
  # return (100.0 * correct / total, loss/total)
  return 100.0 * correct / total      
      
testset=importer()
net=torch.load('ModelNet10')
testloader=torch.utils.data.DataLoader(testset, batch_size=32, shuffle=True,num_workers=4)
accuracy(net,testloader)


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive
0
1
2
3
4
5
6
7
8
9
10
Accuracy of the network on the loader : 96 %


96.32292522596549

In [0]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def plot3DVoxel(voxels):
    fig = plt.figure('Point Cloud 3D Voxelization')
    fig, _ = plt.subplots(figsize=(10, 10))
    plt3d = fig.gca(projection='3d')

    occupied = (voxels == 1)
    free = (voxels == 0)

    # # set the colors of each object
    colors = np.zeros(voxels.shape + (4,))
    colors[free] = [0.1, 0.1, 0.1, 0.1]
    colors[occupied] = [0.8, 0.8, 0.8, 1.0]

    # setting camera angle
    plt3d.set_xlabel('X', fontsize=9)
    plt3d.set_ylabel('Y', fontsize=9)
    plt3d.set_zlabel('Z', fontsize=9)
    plt3d.set_xlim3d(0, voxels.shape[0])
    plt3d.set_ylim3d(0, voxels.shape[1])
    plt3d.set_zlim3d(0, voxels.shape[2])

    # and plot everything
    # plt3d.voxels(occupied | free, facecolors=colors, edgecolor='k', linewidth=0.8)
    plt3d.voxels(occupied, facecolors=colors, edgecolor='k', linewidth=0.8)

    dx, dy, dz = voxels.shape
    x = y= z = 0
    xx = [x, x, x+dx, x+dx, x]
    yy = [y, y+dy, y+dy, y, y]
    kwargs = {'alpha': 1, 'color': 'white'}
    plt3d.plot3D(xx, yy, [z]*5, **kwargs)
    plt3d.plot3D(xx, yy, [z+dz]*5, **kwargs)
    plt3d.plot3D([x, x], [y, y], [z, z+dz], **kwargs)
    plt3d.plot3D([x, x], [y+dy, y+dy], [z, z+dz], **kwargs)
    plt3d.plot3D([x+dx, x+dx], [y+dy, y+dy], [z, z+dz], **kwargs)
    plt3d.plot3D([x+dx, x+dx], [y, y], [z, z+dz], **kwargs)

    plt.show()

volume=torch.Tensor.numpy(testset[1,77])
plot3DVoxel(volume)
net=torch.load('ModelNet40_2.0')
accuracy(net,testloader)